In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
import seaborn as sns
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!pip install streamlit

     |████████████████████████████████| 8.0 MB 8.5 MB/s 
     |████████████████████████████████| 111 kB 66.3 MB/s 
     |████████████████████████████████| 4.3 MB 42.9 MB/s 
     |████████████████████████████████| 170 kB 71.0 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 123 kB 69.1 MB/s 
     |████████████████████████████████| 787 kB 60.1 MB/s 
     |████████████████████████████████| 370 kB 73.0 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=43577ddf4872ab1dd2366272b829f20fdff6786b87aeb0f7193a0b088675f408
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting

In [3]:
!pip install pyngrok

     |████████████████████████████████| 745 kB 7.0 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=8f82228880527e33428c3c2ba57005e2055a6b7fb7f4371f5d97f030837e0022
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import re
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import linear_kernel
import random

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

data_read = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/Beer_Recommendations_EDA_V01.csv')
df = data_read.copy()
df = df.sample(frac=0.10, random_state=1)

# Remove Punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
df['text'] = df['text'].apply(remove_punctuations)

# Make Strings Lowercase
df['text'] = df['text'].str.lower()

# Remove Digits
df['text'] = df['text'].str.replace('\d+', '')

#define lemmatizer
lemmatizer = WordNetLemmatizer()

#remove stopwords
stop = stopwords.words('english')
df['without_stopwords']  = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#lemmatize/tokenize/make everything lowercase/remove punctuation
df['lemmatized_text'] = df['without_stopwords'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))

#remove digitsvb
df['lemmatized_text'] = df['lemmatized_text'].str.replace('\d+', '')

#add some more stopwords
newStopWords = ['beer', 'one', 'would', 'get', 'come', 'also', 'oz', 'could']
stop.extend(newStopWords)
df['lemmatized_text'] = df['lemmatized_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

df['lemmatized_text'] = df['lemmatized_text'].astype('str')


# add some more stopwords
newStopWords = ['lager', 'porter', 'stout', 'brewery', 'ale', 'apa', 'ipa', 'ipas', 'belgian', 'red', 'brown', 'cream', 'black', 'amber',
               'golden', 'much', 'year', 'worth', 'english', 'german', 'american', 'stuff', 'pilsner', 'old', 'barleywine',
               'ri', 'imperial', 'non', 'dipa', 'iipa', 'dipas', 'pils', 'pilsener', 'sam', 'irish', 'brewpub', 'st',
               'adam', 'ml', 'tripel', 'quad', 'pa', 'ibu', 'ibus', 'series', 'bell', 'belgium', 'boston', 'city', 'coors',
               'dead', 'dfh', 'dog', 'dogfish', 'founder', 'im', 'imo', 'lagunitas', 'left', 'nevada', 'rogue', 'samuel', 'sierra',
               'southern', 'pint']
stop.extend(newStopWords)
df['lemmatized_text'] = df['lemmatized_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

nltk.download('averaged_perceptron_tagger')

df['lemmatized_text'] = df['lemmatized_text'].astype('str')

def nouns_adj(text):
    #Given a string of text, tokenize the text and pull out only the nouns and adjectives.
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)]
    return ' '.join(nouns_adj)

df['adjectives/nouns'] = df['lemmatized_text'].apply(nouns_adj)
df = df.dropna()

#create new dataframe
beer_text = df[['beer_id', 'beer_name', 'style', 'brewery_name', 'Name', 'lemmatized_text', 'adjectives/nouns']]

#drop null values
beer_text = df.dropna(axis=0, subset=['adjectives/nouns'])

#group by beer, think of each beer as a "document"
grouped_beer_text = beer_text.groupby(['beer_id', 'brewery_name', 'beer_name', 'Name', 'style'])['adjectives/nouns'].agg(lambda col: ''.join(col))

#df
beer_text = pd.DataFrame(grouped_beer_text)
beer_text = beer_text.reset_index()

# using lemmatized_text, create the corpus
corpus = beer_text['adjectives/nouns']

# FEATURE EXTRACTION via tfidf vectorizer
tfidf_model = TfidfVectorizer(max_features=500,
                             max_df=0.25,
                             min_df=0.01)

tfidf_matrix = tfidf_model.fit_transform(corpus).todense()
print('The shape of the matrix is:', tfidf_matrix.shape)
tfidf_df = pd.DataFrame(tfidf_matrix)
tfidf_df.columns = sorted(tfidf_model.vocabulary_)

# Calculate the cosine similarity of the matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Construct a reverse mapping of indices and beer names, and drop duplicate names, if any
indices = pd.Series(beer_text.index, index=beer_text['Name']).drop_duplicates()

print(type(indices))
print(type(cosine_sim))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
The shape of the matrix is: (1709, 500)
<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>


In [6]:
import pickle
indices.to_pickle('indices')
np.save("cosine_sim.npy", cosine_sim)

In [7]:
beer_text.to_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/beer_text.csv', encoding='utf-8', index=False)

In [8]:
%%writefile content.py
import numpy as np
import pandas as pd



indices = pd.read_pickle('indices')
cosine_sim = np.load("cosine_sim.npy")
beer_text = pd.read_csv('/content/drive/My Drive/LHL_Final_Project/Written_Reviews/beer_text.csv')

import streamlit as st
st.set_page_config(layout="wide") 
st.title('Beer Recommendation App - Content Recommendation')
st.write('This app was built to help you expand your beer horizons')
st.write('Data obtained from https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews')


def content_recommender(title, cosine_sim=cosine_sim, df=beer_text, indices=indices):
    # Obtain the index of the beer that matches the name
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    beer_indices = [i[0] for i in sim_scores]

    
    return beer_text[['style', 'brewery_name', 'beer_name']].iloc[beer_indices]

st.write('What are you in the mood for?')

want = st.selectbox('Select', ['I want a beer for summer, hanging with friends on patios, and barbecueing.',
        'I love the bitter flavour of black coffee or a strong grapefruit.',
        'I love coffee-crisp, purposely burnt marshmallows, and other bold rich flavours.',
        'I love dried fruit, Werther’s Originals, bold flavours, and just a pinch of sweet.',
        'I love scotch, campfires, and smoked meats.',
        "I love a glass of orange juice and I never says 'when' when the waiter is adding pepper to my meal.",
        'I drink green tea instead of coffee, love cucumber in my water, and am semi-addicted to sour candies'])
if 'barbecue' in want:
  test_beer = 'Molson Coors Canada Molson Canadian Lager'
elif 'marshmallows' in want:
  test_beer = 'Guinness Ltd Guinness Extra Stout Original'
elif 'chugging' in want:
  test_beer = 'Coors Brewing Company MolsonCoors Blue Moon Belgian White'
elif 'grapefruit' in want:
  test_beer = 'Dogfish Head Craft Brewery 90 Minute IPA'
elif 'Original' in want:
  test_beer = 'Samuel Smith Old Brewery Tadcaster Nut Brown Ale'
elif 'smoked' in want:
  test_beer = 'Alaskan Brewing Co Alaskan Smoked Porter'
elif 'cucumber' in want:
  test_beer = 'Westbrook Brewing Co Gose'
else:
  test_beer = "I'm not a beer person"


st.write(content_recommender(test_beer))

Writing content.py


In [9]:
!ls

content.py  cosine_sim.npy  drive  indices  sample_data


# Insert ngrok authtoken

In [10]:
!ngrok authtoken XXXXXXXXXXXXXXXXXXXXXXXXXXX

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [11]:
from pyngrok import ngrok

In [12]:
!streamlit run --server.port 80 content.py&>/dev/null&

In [15]:
#publ_url = ngrok.connect(port='8501')
publ_url = ngrok.connect(port='80')

In [16]:
publ_url 

<NgrokTunnel: "http://4a1b-34-125-70-129.ngrok.io" -> "http://localhost:80">

In [17]:
ngrok.kill()